# 分组与聚合操作

在Excel中，我们经常使用"分类汇总"和"数据透视表"来实现分组统计。Pandas的groupby提供了更强大的分组功能，让我们可以进行更复杂的分析。

## 基础分组操作

In [1]:
# 导入所需的库
import pandas as pd
import numpy as np

# 读取数据
df_students = pd.read_csv('data/student_info.csv')
df_orders = pd.read_csv('data/order_info.csv')
df_products = pd.read_csv('data/product_info.csv')
df_order_details = pd.read_csv('data/order_details.csv')


## 基础分组操作

### 1. 单字段分组

In [2]:
# 最基本的分组操作，类似于Excel中的分类汇总
# 按'商品类别'分组，统计每个类别的订单金额总和
basic_group = df_orders.groupby('商品类别')['订单金额'].sum()
print("基本分组统计（按商品类别汇总销售额）：")  # 输出基本分组统计的标题
print(basic_group)  # 打印每个商品类别的总销售额

# 查看分组的基本信息
print("\n分组信息：")  # 输出分组信息的标题
group_info = df_orders.groupby('商品类别')  # 创建一个按'商品类别'分组的对象
print(f"分组数量: {len(group_info)}")  # 打印分组数量
print(f"分组项: {list(group_info.groups.keys())}")  # 打印分组项的名称列表

# 分组后的描述性统计，类似于Excel中的描述统计功能
group_desc = df_orders.groupby('商品类别')['订单金额'].describe()
print("\n各类别订单金额描述性统计：")  # 输出描述性统计的标题
print(group_desc)  # 打印每个商品类别的订单金额的描述性统计信息


基本分组统计（按商品类别汇总销售额）：
商品类别
图书       111399.04
学习用品      38804.78
服装       242474.24
生活用品     139853.65
电子产品    2986024.71
运动器材     351649.84
食品        62320.16
Name: 订单金额, dtype: float64

分组信息：
分组数量: 7
分组项: ['图书', '学习用品', '服装', '生活用品', '电子产品', '运动器材', '食品']

各类别订单金额描述性统计：
      count         mean          std     min        25%       50%        75%  \
商品类别                                                                            
图书    448.0   248.658571   168.770425   20.50   110.9500   201.015    366.090   
学习用品  453.0    85.661766    57.940069    6.11    37.6500    73.140    120.400   
服装    474.0   511.549030   358.580881   35.85   225.4200   409.695    760.575   
生活用品  416.0   336.186659   228.130880   20.25   147.2075   293.825    462.960   
电子产品  403.0  7409.490596  5687.776020  132.48  2830.3150  6237.680  10177.450   
运动器材  427.0   823.535925   566.752303   50.45   350.8400   688.200   1244.845   
食品    407.0   153.120786   112.010836   10.40    66.4250   123.480    213.330   


多字段分组

In [3]:
# 按多个字段分组，类似于Excel中的多级分类汇总
multi_group = df_orders.groupby(['商品类别', '年级'])['订单金额'].agg([
    ('订单数', 'count'),  # 计算每个分组的订单数量
    ('总金额', 'sum'),    # 计算每个分组的订单金额总和
    ('平均金额', 'mean')  # 计算每个分组的订单金额平均值
]).round(2)  # 将结果四舍五入到小数点后两位

print("多字段分组统计：")  # 输出多字段分组统计的标题
print(multi_group)  # 打印多字段分组统计的结果

# 按多个字段分组，对不同的列应用不同的聚合函数进行统计分析
multi_stats = df_orders.groupby(['商品类别', '年级']).agg({
    '订单金额': ['sum', 'mean'],  # 对'订单金额'列进行求和和求平均
    '购买数量': 'sum',            # 对'购买数量'列进行求和
    '优惠金额': ['sum', 'mean']   # 对'优惠金额'列进行求和和求平均
}).round(2)  # 将结果四舍五入到小数点后两位

print("\n多字段多指标统计：")  # 输出多字段多指标统计的标题
print(multi_stats)  # 打印多字段多指标统计的结果


多字段分组统计：
         订单数        总金额     平均金额
商品类别 年级                         
图书   大一  119   28058.13   235.78
     大三  105   25424.55   242.14
     大二   91   24642.82   270.80
     大四  133   33273.54   250.18
学习用品 大一  103    8780.44    85.25
     大三  130   11104.98    85.42
     大二  101    8023.15    79.44
     大四  119   10896.21    91.56
服装   大一  115   57876.08   503.27
     大三  116   62729.98   540.78
     大二   96   48709.63   507.39
     大四  147   73158.55   497.68
生活用品 大一   93   31246.15   335.98
     大三   92   29014.61   315.38
     大二   92   30295.76   329.30
     大四  139   49297.13   354.66
电子产品 大一   96  790696.65  8236.42
     大三   95  673539.79  7089.89
     大二   86  598537.27  6959.74
     大四  126  923251.00  7327.39
运动器材 大一  100   79125.40   791.25
     大三   95   79679.66   838.73
     大二  105   94028.11   895.51
     大四  127   98816.67   778.08
食品   大一  108   15844.35   146.71
     大三   94   15025.19   159.84
     大二   94   14793.78   157.38
     大四  111   16656.84   150.06



分组统计（agg）

In [6]:
# 使用 agg 方法进行复杂的分组统计
advanced_stats = df_orders.groupby('商品类别').agg({
    '订单金额': [  # 对 '订单金额' 列进行多种统计
        ('总销售额', 'sum'),  # 计算每个商品类别的订单金额总和，并重命名为 "总销售额"
        ('平均订单额', 'mean'),  # 计算每个商品类别的平均订单金额，并重命名为 "平均订单额"
        ('订单数', 'count'),  # 计算每个商品类别的订单数量，并重命名为 "订单数"
        ('最大订单', 'max'),  # 计算每个商品类别的最大订单金额，并重命名为 "最大订单"
        ('最小订单', 'min')  # 计算每个商品类别的最小订单金额，并重命名为 "最小订单"
    ],
    '购买数量': [  # 对 '购买数量' 列进行统计
        ('总数量', 'sum'),  # 计算每个商品类别的购买数量总和，并重命名为 "总数量"
        ('平均数量', 'mean')  # 计算每个商品类别的平均购买数量，并重命名为 "平均数量"
    ]
}).round(2)  # 将结果四舍五入到小数点后两位

print("复杂分组统计：")  # 输出复杂分组统计的标题
print(advanced_stats)  # 打印复杂分组统计的结果


复杂分组统计：
            订单金额                                  购买数量      
            总销售额    平均订单额  订单数      最大订单    最小订单   总数量  平均数量
商品类别                                                        
图书     111399.04   248.66  448    742.85   20.50  1320  2.95
学习用品    38804.78    85.66  453    246.95    6.11  1389  3.07
服装     242474.24   511.55  474   1498.80   35.85  1435  3.03
生活用品   139853.65   336.19  416    994.30   20.25  1295  3.11
电子产品  2986024.71  7409.49  403  24901.15  132.48  1204  2.99
运动器材   351649.84   823.54  427   2465.80   50.45  1273  2.98
食品      62320.16   153.12  407    498.50   10.40  1190  2.92


分组转换（transform）

In [4]:
# 使用 transform 方法进行组内计算（在 Excel 中需要用复杂的公式实现）
# 计算每个订单金额在所属商品类别总金额中所占的比例
df_orders['类别总金额'] = df_orders.groupby('商品类别')['订单金额'].transform('sum')  
# 对 '商品类别' 分组，计算每个类别的订单金额总和，并将结果分配到每个订单中

df_orders['占类别比例'] = (df_orders['订单金额'] / df_orders['类别总金额']).round(4)  
# 计算每个订单金额占该类别总金额的比例，并将结果保留到小数点后四位

# 计算每个订单金额与其类别的平均金额之间的差异
df_orders['类别平均金额'] = df_orders.groupby('商品类别')['订单金额'].transform('mean')  
# 对 '商品类别' 分组，计算每个类别的订单金额平均值，并将结果分配到每个订单中

df_orders['与平均值差异'] = (df_orders['订单金额'] - df_orders['类别平均金额']).round(2)  
# 计算每个订单金额与类别平均金额的差异，并将结果保留到小数点后两位

print("分组转换结果示例：")  
# 输出分组转换结果示例的标题

print(df_orders[['商品类别', '订单金额', '类别总金额', '占类别比例', '类别平均金额', '与平均值差异']].head())  
# 打印数据框中选定的列，展示前几行的分组转换结果

分组转换结果示例：
   商品类别     订单金额      类别总金额   占类别比例      类别平均金额  与平均值差异
0  运动器材  1102.11  351649.84  0.0031  823.535925  278.57
1    食品    88.35   62320.16  0.0014  153.120786  -64.77
2  生活用品   108.48  139853.65  0.0008  336.186659 -227.71
3    图书    79.72  111399.04  0.0007  248.658571 -168.94
4  生活用品   404.97  139853.65  0.0029  336.186659   68.78


### 高级分组操作

#### 1. 自定义聚合函数

In [9]:
# 定义自定义聚合函数（实现 Excel 中难以完成的复杂计算）
def value_range(x):
    """计算数值范围"""
    return x.max() - x.min()  
    # 计算传入数据的最大值与最小值之差，得到数值范围

def top_3_mean(x):
    """计算前3名的平均值"""
    return x.nlargest(3).mean()  
    # 取数据中最大的3个值并计算它们的平均值

# 使用自定义函数进行分组统计
custom_agg = df_orders.groupby('商品类别')['订单金额'].agg([
    ('数值范围', value_range),  # 使用自定义函数 value_range 计算数值范围
    ('前3名平均', top_3_mean)   # 使用自定义函数 top_3_mean 计算前3名的平均值
]).round(2)  # 将结果保留到小数点后两位

print("自定义聚合统计：")
print(custom_agg)
print("自定义聚合统计：")  
# 输出自定义聚合统计的标题

print(custom_agg)  
# 打印自定义聚合统计的结果

自定义聚合统计：
          数值范围     前3名平均
商品类别                    
图书      722.35    734.50
学习用品    240.84    243.42
服装     1462.95   1477.68
生活用品    974.05    979.92
电子产品  24768.67  24630.33
运动器材   2415.35   2443.15
食品      488.10    497.83
自定义聚合统计：
          数值范围     前3名平均
商品类别                    
图书      722.35    734.50
学习用品    240.84    243.42
服装     1462.95   1477.68
生活用品    974.05    979.92
电子产品  24768.67  24630.33
运动器材   2415.35   2443.15
食品      488.10    497.83


多个聚合函数同时使用

In [10]:
# 综合分析（相当于 Excel 中多个统计表的组合）
comprehensive_analysis = df_orders.groupby(['商品类别', '年级']).agg({
    '订单金额': [
        ('总额', 'sum'),  # 计算订单金额的总和，并命名为 "总额"
        ('均值', 'mean'),  # 计算订单金额的平均值，并命名为 "均值"
        ('数量', 'count'),  # 计算订单数，并命名为 "数量"
        ('范围', value_range),  # 使用自定义函数计算数值范围，并命名为 "范围"
        ('前3均值', top_3_mean)  # 使用自定义函数计算前3名的平均值，并命名为 "前3均值"
    ],
    '购买数量': [
        ('总量', 'sum'),  # 计算购买数量的总和，并命名为 "总量"
        ('均量', 'mean')  # 计算购买数量的平均值，并命名为 "均量"
    ]
}).round(2)  
# 将所有结果保留到小数点后两位

print("综合统计分析：")  
# 输出综合统计分析的标题

print(comprehensive_analysis)  
# 打印综合统计分析的结果

综合统计分析：
              订单金额                                   购买数量      
                总额       均值   数量        范围      前3均值   总量    均量
商品类别 年级                                                        
图书   大一   28058.13   235.78  119    697.60    671.57  333  2.80
     大三   25424.55   242.14  105    703.04    719.60  320  3.05
     大二   24642.82   270.80   91    621.91    656.33  274  3.01
     大四   33273.54   250.18  133    722.25    704.58  393  2.95
学习用品 大一    8780.44    85.25  103    220.91    227.12  316  3.07
     大三   11104.98    85.42  130    230.06    235.08  416  3.20
     大二    8023.15    79.44  101    236.05    224.27  295  2.92
     大四   10896.21    91.56  119    240.84    237.72  362  3.04
服装   大一   57876.08   503.27  115   1258.54   1289.10  346  3.01
     大三   62729.98   540.78  116   1462.95   1417.68  369  3.18
     大二   48709.63   507.39   96   1413.39   1447.63  275  2.86
     大四   73158.55   497.68  147   1440.30   1431.25  445  3.03
生活用品 大一   31246.15   335.98   93

实际应用示例

In [13]:
def sales_analysis(group):
    """综合销售分析"""
    return pd.Series({
        '订单数': len(group),  # 计算订单数
        '总金额': group['订单金额'].sum(),  # 计算总销售金额
        '平均金额': group['订单金额'].mean(),  # 计算平均订单金额
        '最大订单': group['订单金额'].max(),  # 计算最大订单金额
        '平均数量': group['购买数量'].mean(),  # 计算平均购买数量
        '优惠比例': (group['优惠金额'].sum() / group['订单金额'].sum() * 100).round(2),  # 计算优惠金额占订单金额的百分比
        '前3均值': group['订单金额'].nlargest(3).mean()  # 计算订单金额中最大的3个值的平均
    })

# 使用 apply 进行分组计算，并排除分组列
sales_report = df_orders.groupby(['商品类别', '年级'], group_keys=False).apply(sales_analysis)

print("综合销售分析报表：")
print(sales_report)

综合销售分析报表：
           订单数        总金额         平均金额      最大订单      平均数量  优惠比例          前3均值
商品类别 年级                                                                       
图书   大一  119.0   28058.13   235.782605    718.10  2.798319  5.18    671.566667
     大三  105.0   25424.55   242.138571    742.55  3.047619  3.93    719.600000
     大二   91.0   24642.82   270.800220    665.65  3.010989  4.77    656.333333
     大四  133.0   33273.54   250.176992    742.85  2.954887  4.59    704.583333
学习用品 大一  103.0    8780.44    85.246990    233.15  3.067961  5.70    227.116667
     大三  130.0   11104.98    85.422923    238.90  3.200000  5.96    235.083333
     大二  101.0    8023.15    79.437129    244.40  2.920792  5.88    224.266667
     大四  119.0   10896.21    91.564790    246.95  3.042017  5.71    237.716667
服装   大一  115.0   57876.08   503.270261   1309.10  3.008696  5.62   1289.100000
     大三  116.0   62729.98   540.775690   1498.80  3.181034  5.12   1417.683333
     大二   96.0   48709.63   507.391979   1

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20912\3356690160.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sales_report = df_orders.groupby(['商品类别', '年级'], group_keys=False).apply(sales_analysis)
